In [9]:
import random
import numpy as np
import cv2
import os
import torch
import torch.utils.data as data
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision.transforms import ToTensor, Normalize, Compose, ToPILImage
from os.path import join
from os import listdir
from torchsummary import summary
import time
import argparse
import models.models.DnCNN as DnCNN, models.models.ResNet as ResNet, models.models.RFDN as RFDN
import models.models.DRLN as DRLN, models.models.pix2pix as pix2pix, models.models.pix2pix2 as pix2pix2
import models.utils.vgg_loss, models.utils.vgg_perceptual_loss
import matplotlib.pyplot as plt
from models.utils.param import seed_everything

model_list = ['DnCNN', 'ResNet18', 'RFDN', 'DRLN', 'pix2pix', 'pix2pix2']

sel_model = model_list[5]
pth_num =2
noise = False
val = 0.001
if noise: pth = '.pth'
else: pth = '_clean.pth' 
if sel_model=='pix2pix' or sel_model=='pix2pix2': pth='_g.pth'
save_record = False # 기록용 폴더에 있으면
pth_dir ='./save/best_'+sel_model+'_model'+str(pth_num)+pth
if save_record:
    pth_dir ='./기록용/'+sel_model+'/'+'best_'+sel_model+'_model'+str(pth_num)+pth
# load_loss_pth ='./ResNet18_model2.pth'

display_number = 20                           # plt로 디스플레이할 이미지의 개수
residual_output_paths = '/content/residuals'           # 잔차 이미지를 저장할 폴더
noisy_image_paths = '/local_datasets/MLinP/train/scan'
clean_image_paths = '/local_datasets/MLinP/train/clean'

random.seed(3002)
output = False                                         # residual을 파일로 출력할지 여부


def load_img(filepath):
    img = cv2.imread(filepath)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

def cvt_to_image(image):
    image = image.cpu().squeeze(0)
    # image = torch.clamp(image, 0, 1)  # 이미지 값을 0과 1 사이로 클램핑
    image = ToPILImage()(image*0.5+0.5)
    return image

class CustomDatasetTest(data.Dataset):
    def __init__(self, noisy_image_paths, clean_image_paths, transform=None):
        self.noisy_image_paths = [join(noisy_image_paths, x) for x in listdir(noisy_image_paths)]
        self.clean_image_paths = [join(clean_image_paths, x) for x in listdir(clean_image_paths)]
        self.transform = transform

    def __len__(self):
        return len(self.noisy_image_paths)

    def __getitem__(self, index):
        
        noisy_image_path = self.noisy_image_paths[index]
        noisy_image = load_img(self.noisy_image_paths[index])
        clean_image_path = self.clean_image_paths[index]
        clean_image = load_img(self.clean_image_paths[index])
        
        if self.transform:
            noisy_image = self.transform(noisy_image)
            clean_image = self.transform(clean_image)

        return noisy_image, noisy_image_path, clean_image, clean_image_path

class BilateralBlur(object):
    def __init__(self, output_size):
        assert isinstance(output_size, int)
        if isinstance(output_size, int):
            self.output_size = (output_size, output_size)
        else:
            assert len(output_size) == 2
            self.output_size = output_size

    def __call__(self,sample):
        image = sample
        h, w = image.shape[:2]

        return cv2.bilateralFilter(image,-1,10,5)

if sel_model == 'DnCNN':
    model = DnCNN.DnCNN()
elif sel_model == 'ResNet18':
    model = ResNet.ResNet18()
elif sel_model == 'RFDN':
    model = RFDN.RFDN()
elif sel_model == 'DRLN':
    model = DRLN.DRLN()
elif sel_model == 'pix2pix':
    model = pix2pix.Generator()
elif sel_model == 'pix2pix2':
    model = pix2pix2.GeneratorUNet()
    model_D = pix2pix2.Discriminator()

device = torch.device('cuda' if torch.cuda.is_available() else 'mps:0' if torch.backends.mps.is_available() else 'cpu')
# device = torch.device('cpu')
model.load_state_dict(torch.load(pth_dir, map_location='cpu'))
model.eval()

# GPU 사용 여부 확인
model.to(device)

test_transform = Compose([
    # BilateralBlur(512),
    ToTensor(),
    Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# 데이터셋 로드 및 전처리
dataset = CustomDatasetTest(noisy_image_paths, clean_image_paths ,transform=test_transform)
# val 분할
train_size = int(len(dataset)*(1-val))
val_size = int(len(dataset)*(val))
train_dataset, val_dataset = random_split(dataset,[train_size,val_size])
# 데이터 로더 설정
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)
tot_mae = 0.0






for noisy_image, _, clean_image, _  in val_loader:
    noisy_image = noisy_image.to(device)
# random_idx = random.sample(range(len(noisy_dataset)), display_number)
# for idx in random_idx:
#     noisy_image, _, clean_image, _ = noisy_dataset.__getitem__(idx)
    noisy_image = noisy_image.to(device)
    if noise:
        pred_noise = model(noisy_image)
        denoised_image = noisy_image - pred_noise  
    else:
        denoised_image = model(noisy_image)
        pred_noise = noisy_image - denoised_image 
    # denoised_image를 CPU로 이동하여 이미지 저장
    residual_image = noisy_image.cpu() - clean_image

    denoised_image = cvt_to_image(denoised_image)
    noisy_image = cvt_to_image(noisy_image)
    clean_image = cvt_to_image(clean_image)
    residual_image = cvt_to_image(residual_image)
    pred_noise = cvt_to_image(pred_noise)


    plt.figure(figsize=(20,13))
    plt.subplot(2,3,1)
    plt.imshow(noisy_image)
    plt.axis('off')
    plt.subplot(2,3,2)
    plt.imshow(clean_image)
    plt.axis('off')
    plt.subplot(2,3,3)
    plt.imshow(denoised_image)
    plt.axis('off')
    plt.subplot(2,3,4)
    plt.imshow(residual_image)
    plt.axis('off')
    plt.subplot(2,3,5)
    plt.imshow(pred_noise)
    plt.axis('off')
    plt.show()

# avg_residual = torch.mean(residuals,axis=0)

RuntimeError: PytorchStreamReader failed reading zip archive: failed finding central directory

In [13]:
a = 'asd_d'

print(a)
print(a.replace('_d','_g'))
print(a)
print(a.replace('_d','_g'))

asd_d
asd_g
asd_d
asd_g


In [2]:
import random
import numpy as np
import cv2
import os
import torch
import torch.utils.data as data
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision.transforms import ToTensor, Normalize, Compose, ToPILImage
import torchvision.transforms as tf
from os.path import join
from os import listdir
from torchsummary import summary
import time
import argparse
import models.models.DnCNN as DnCNN, models.models.ResNet as ResNet, models.models.RFDN as RFDN, models.models.DRLN as DRLN, models.models.pix2pix as pix2pix
# from models.utils.param import param_check, seed_everything
from models.utils.param import seed_everything
import models.utils.vgg_loss, models.utils.vgg_perceptual_loss
import matplotlib.pyplot as plt

model_list = ['DnCNN', 'ResNet18', 'RFDN', 'DRLN', 'pix2pix']

sel_model = model_list[2]
pth_num =13
noise = True
val = 0.001
if noise: pth = '.pth'
else: pth = '_clean.pth' 
if sel_model=='pix2pix': pth='_g.pth'
save_record = False # 기록용 폴더에 있으면
pth_dir ='./save/best_'+sel_model+'_model'+str(pth_num)+pth
if save_record:
    pth_dir ='./기록용/'+sel_model+'/'+'best_'+sel_model+'_model'+str(pth_num)+pth
# load_loss_pth ='./ResNet18_model2.pth'

display_number = 10                        # plt로 디스플레이할 이미지의 개수
residual_output_paths = '/content/residuals'           # 잔차 이미지를 저장할 폴더
noisy_image_paths = '/local_datasets/MLinP/train/scan'
clean_image_paths = '/local_datasets/MLinP/train/clean'

seed_everything(42)
output = False                                         # residual을 파일로 출력할지 여부


def load_img(filepath):
    img = cv2.imread(filepath)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

def cvt_to_image(image):
    image = image.cpu().squeeze(0)
     # denoised_image = torch.clamp(denoised_image, 0, 1)  # 이미지 값을 0과 1 사이로 클램핑
    image = ToPILImage()(image*0.5+0.5)
    image = np.array(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
    image = image[:, :, 0]
    return image

class CustomDatasetTest(data.Dataset):
    def __init__(self, noisy_image_paths, clean_image_paths, transform=None):
        self.noisy_image_paths = [join(noisy_image_paths, x) for x in listdir(noisy_image_paths)]
        self.clean_image_paths = [join(clean_image_paths, x) for x in listdir(clean_image_paths)]
        self.transform = transform

    def __len__(self):
        return len(self.noisy_image_paths)

    def __getitem__(self, index):
        
        noisy_image_path = self.noisy_image_paths[index]
        noisy_image = load_img(self.noisy_image_paths[index])
        clean_image_path = self.clean_image_paths[index]
        clean_image = load_img(self.clean_image_paths[index])
        
        if self.transform:
            noisy_image = self.transform(noisy_image)
            clean_image = self.transform(clean_image)

        return noisy_image, noisy_image_path, clean_image, clean_image_path

# Transform Definition
class RGBtoYcrcb(object):
    def __init__(self, ):
        return 

    def __call__(self,sample):
        image = sample
        image = cv2.cvtColor(image,cv2.COLOR_RGB2YCrCb)
        return image

if sel_model == 'DnCNN':
    model = DnCNN.DnCNN()
elif sel_model == 'ResNet18':
    model = ResNet.ResNet18()
elif sel_model == 'RFDN':
    model = RFDN.RFDN()
elif sel_model == 'DRLN':
    model = DRLN.DRLN()
elif sel_model == 'pix2pix':
    model = pix2pix.Generator()

device = torch.device('cuda' if torch.cuda.is_available() else 'mps:0' if torch.backends.mps.is_available() else 'cpu')
# device = torch.device('cpu')
model.load_state_dict(torch.load(pth_dir, map_location='cpu'))
model.eval()

# GPU 사용 여부 확인
model.to(device)

test_transform = Compose([
    # BilateralBlur(512),
    ToTensor(),
    Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])
def tensor_to_yuv(image):
    image = torch.clamp(image, 0, 1)  # 이미지 값을 0과 1 사이로 클램핑
    image = ToPILImage()(image)
    image = np.array(image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
    return

def cal_mae_loss(p,t, y=False):
    if not y:
        p = p[:, :, 0]
        t = t[:, :, 0]
    return abs(np.mean(p.flatten()) - np.mean(t.flatten()))

# 데이터셋 로드 및 전처리
dataset = CustomDatasetTest(noisy_image_paths, clean_image_paths ,transform=test_transform)
# val 분할
train_size = int(len(dataset)*(1-val))
val_size = int(len(dataset)*(val))
train_dataset, val_dataset = random_split(dataset,[train_size,val_size])
# 데이터 로더 설정
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)
tot_mae = 0.0






for noisy_image, _, clean_image, _  in val_loader:
    noisy_image = noisy_image.to(device)
    if noise:
        pred_noise = model(noisy_image)
        denoised_image = noisy_image - pred_noise  
    else:
        denoised_image = model(noisy_image)
        pred_noise = noisy_image - denoised_image 
    # denoised_image를 CPU로 이동하여 이미지 저장
    residual_image = noisy_image.cpu() - clean_image
    clean_image = cvt_to_image(clean_image)
    denoised_image = cvt_to_image(denoised_image)
    noisy_image = cvt_to_image(noisy_image)
    residual_image = cvt_to_image(residual_image)
    pred_noise = cvt_to_image(pred_noise)
    mae_loss = cal_mae_loss(denoised_image,clean_image,True)
    tot_mae += mae_loss
    print('This MAE: ',mae_loss)
    if False:
        plt.figure(figsize=(20,13))
        plt.subplot(2,3,1)
        plt.imshow(noisy_image, cmap='gray')
        plt.axis('off')
        plt.subplot(2,3,2)
        plt.imshow(clean_image, cmap='gray')
        plt.axis('off')
        plt.subplot(2,3,3)
        plt.imshow(denoised_image, cmap='gray')
        plt.axis('off')
        plt.subplot(2,3,4)
        plt.imshow(residual_image, cmap='gray')
        plt.axis('off')
        plt.subplot(2,3,5)
        plt.imshow(pred_noise, cmap='gray')
        plt.axis('off')
        plt.show()
print(len(val_loader),'개의', 'total MAE: ', tot_mae/len(val_loader))
# avg_residual = torch.mean(residuals,axis=0)

This MAE:  2.743450164794922
This MAE:  2.7954139709472656
This MAE:  2.7238540649414062
This MAE:  4.730930328369141
This MAE:  2.3360748291015625
This MAE:  1.4974746704101562
This MAE:  7.538585662841797
This MAE:  4.353954315185547
This MAE:  8.776409149169922
This MAE:  9.777458190917969
10 개의 total MAE:  4.727360534667969


In [7]:
import random
import numpy as np
import cv2
import os
import torch
import torch.utils.data as data
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision.transforms import ToTensor, Normalize, Compose, ToPILImage
import torchvision.transforms as tf
from os.path import join
from os import listdir
from torchsummary import summary
import time
import argparse
import models.models.DnCNN as DnCNN, models.models.ResNet as ResNet, models.models.RFDN as RFDN, models.models.DRLN as DRLN, models.models.pix2pix as pix2pix
# from models.utils.param import param_check, seed_everything
from models.utils.param import seed_everything
import models.utils.vgg_loss, models.utils.vgg_perceptual_loss
import matplotlib.pyplot as plt

model_list = ['DnCNN', 'ResNet18', 'RFDN', 'DRLN', 'pix2pix']

sel_model = model_list[2]
pth_num =13
noise = True
val = 1
if noise: pth = '.pth'
else: pth = '_clean.pth' 
if sel_model=='pix2pix': pth='_g.pth'
save_record = False # 기록용 폴더에 있으면
pth_dir ='./save/best_'+sel_model+'_model'+str(pth_num)+pth
if save_record:
    pth_dir ='./기록용/'+sel_model+'/'+'best_'+sel_model+'_model'+str(pth_num)+pth
# load_loss_pth ='./ResNet18_model2.pth'

display_number = 10                        # plt로 디스플레이할 이미지의 개수
residual_output_paths = '/content/residuals'           # 잔차 이미지를 저장할 폴더
noisy_image_paths = '/local_datasets/MLinP/train/scan'
clean_image_paths = '/local_datasets/MLinP/train/clean'

seed_everything(42)
output = False                                         # residual을 파일로 출력할지 여부


def load_img(filepath):
    img = cv2.imread(filepath)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

def cvt_to_image(image):
    image = image.cpu().squeeze(0)
     # denoised_image = torch.clamp(denoised_image, 0, 1)  # 이미지 값을 0과 1 사이로 클램핑
    image = ToPILImage()(image*0.5+0.5)
    image = np.array(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
    image = image[:, :, 0]
    return image

def tensor_to_yuv(image):
    image = torch.clamp(image, 0, 1)  # 이미지 값을 0과 1 사이로 클램핑
    image = ToPILImage()(image)
    image = np.array(image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
    return

def cal_mae_loss(p,t, y=False):
    if not y:
        p = p[:, :, 0]
        t = t[:, :, 0]
    return abs(np.mean(p.flatten()) - np.mean(t.flatten()))

# 데이터셋 로드 및 전처리
dataset = CustomDatasetTest(noisy_image_paths, clean_image_paths ,transform=test_transform)
# val 분할
train_size = int(len(dataset)*(1-val))
val_size = int(len(dataset)*(val))
train_dataset, val_dataset = random_split(dataset,[train_size,val_size])
# 데이터 로더 설정
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)
tot_mae = 0.0



for noisy_image, _, clean_image, _  in val_loader:
    noisy_image = noisy_image
    if noise:
        pred_noise = noisy_image
        denoised_image = noisy_image - pred_noise  
    else:
        denoised_image = noisy_image
        pred_noise = noisy_image - denoised_image 
    # denoised_image를 CPU로 이동하여 이미지 저장
    residual_image = noisy_image.cpu() - clean_image
    clean_image = cvt_to_image(clean_image)
    denoised_image = cvt_to_image(denoised_image)
    noisy_image = cvt_to_image(noisy_image)
    residual_image = cvt_to_image(residual_image)
    pred_noise = cvt_to_image(pred_noise)
    mae_loss = cal_mae_loss(denoised_image,clean_image,True)
    tot_mae += mae_loss
    print('This MAE: ',mae_loss)
    if False:
        plt.figure(figsize=(20,13))
        plt.subplot(2,3,1)
        plt.imshow(noisy_image, cmap='gray')
        plt.axis('off')
        plt.subplot(2,3,2)
        plt.imshow(clean_image, cmap='gray')
        plt.axis('off')
        plt.subplot(2,3,3)
        plt.imshow(denoised_image, cmap='gray')
        plt.axis('off')
        plt.subplot(2,3,4)
        plt.imshow(residual_image, cmap='gray')
        plt.axis('off')
        plt.subplot(2,3,5)
        plt.imshow(pred_noise, cmap='gray')
        plt.axis('off')
        plt.show()
print(len(val_loader),'개의', 'total MAE: ', tot_mae/len(val_loader))
# avg_residual = torch.mean(residuals,axis=0)

This MAE:  66.36465072631836
This MAE:  33.74124526977539
This MAE:  74.45724868774414
This MAE:  69.57144927978516
This MAE:  56.32483673095703
This MAE:  37.40028381347656
This MAE:  121.58252334594727
This MAE:  96.38151931762695
This MAE:  110.15225982666016
This MAE:  98.10154724121094
This MAE:  50.918758392333984
This MAE:  46.80157470703125
This MAE:  64.29006958007812
This MAE:  37.73825454711914
This MAE:  96.780029296875
This MAE:  70.91676330566406
This MAE:  16.417118072509766
This MAE:  30.266220092773438
This MAE:  98.06576538085938
This MAE:  22.142471313476562
This MAE:  28.65526580810547
This MAE:  84.20731735229492
This MAE:  4.663394927978516
This MAE:  44.4543571472168
This MAE:  12.01611328125
This MAE:  109.49263381958008
This MAE:  94.89456939697266
This MAE:  24.884483337402344
This MAE:  95.43405151367188
This MAE:  52.75619888305664
This MAE:  49.66835403442383
This MAE:  27.184375762939453
This MAE:  77.64626693725586
This MAE:  97.30618667602539
This MAE:  